In [2]:
pip install tensorflow tensorflow-hub


In [3]:
pip install transformers


In [4]:
import pandas as pd

In [5]:
train_df=pd.read_csv('/content/drive/MyDrive/Train.csv')
train_texts=train_df['Text'].values
train_labels=train_df['Label'].values

test_df=pd.read_csv('/content/drive/MyDrive/Test.csv')
test_texts=test_df['Text'].values
test_labels=test_df['Label'].values

In [6]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.metrics import accuracy_score

In [ ]:
import torch
from transformers import LongformerTokenizer, LongformerForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report

# Replace this with your data loading process
# Example: Assume 'texts' contains your input text data and 'labels' contains corresponding class labels
texts = train_texts.tolist()
labels = train_labels.tolist() # Example labels (0, 1, 2, etc.)

# Splitting data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Initialize Longformer tokenizer and model
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=len(set(labels)))

# Tokenizing input texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Creating PyTorch tensors for training and test sets
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_labels)
)
test_dataset = TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(test_labels)
)

# Set batch size and create data loaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Set device to GPU if available
device = torch.device( 'cpu')
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Evaluation
    model.eval()
    predictions, true_labels = [], []
    for batch in test_loader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            logits = model(input_ids=input_ids, attention_mask=attention_mask)

        preds = torch.argmax(logits.logits, dim=1)
        predictions.extend(preds.cpu().detach().numpy())
        true_labels.extend(labels.cpu().detach().numpy())

    print(classification_report(true_labels, predictions))


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
